## Цифровой Прорыв 2022, Чемпионат Ярославской Области
Выполнила Шушпанова Мария.

Загрузим необходимые для работы библиотеки, инициализируем датасеты для обучения и теста. Для удобства загружаем уже категоризованные значения колонок из датасетов. Применяем функцию для очистки датасета.

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import lightgbm as lgm
from sklearn.metrics import recall_score
from tqdm import tqdm, trange
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_dataset_test.csv')

In [3]:
religion = pd.read_csv('religion.csv', sep=';', encoding='cp1251')
sex = pd.read_csv('sex.csv', sep=';', encoding='cp1251')
family = pd.read_csv('family.csv', sep=';', encoding='cp1251')
education = pd.read_csv('education.csv',  sep=';', encoding='cp1251')
ethnos = pd.read_csv('ethnos.csv',  sep=';', encoding='cp1251')
profession = pd.read_csv('profession.csv',  sep=';', encoding='cp1251')
nationality = pd.read_csv('nationality.csv',  sep=';', encoding='cp1251')

In [4]:
def get_clean_df(df: pd.DataFrame, df_dop1: pd.DataFrame, df_dop2: pd.DataFrame, df_dop3: pd.DataFrame, df_dop4: pd.DataFrame,
                df_dop5: pd.DataFrame, df_dop6: pd.DataFrame, df_dop7: pd.DataFrame) -> pd.DataFrame:
    

    df['Частота пасс кур'] = df['Частота пасс кур'].apply(lambda x: str(x).replace('1-2 раза в неделю', '1').replace('3-6 раз в неделю', '2').replace('не менее 1 раза в день', '3').replace('2-3 раза в день', '4').replace('4 и более раз в день', '5').replace('nan', '-1'))
    df['Частота пасс кур'] = df['Частота пасс кур'].astype(int)
    
    df['Алкоголь'] = df['Алкоголь'].apply(lambda x: str(x).replace('употребляю в настоящее время', '1').replace('никогда не употреблял', '0').replace('ранее употреблял', '2').replace('nan', '-1'))
    df['Алкоголь'] = df['Алкоголь'].astype(int)
    
    df['Статус Курения'] = df['Статус Курения'].apply(lambda x: str(x).replace('Никогда не курил(а)', '0').replace('Никогда не курил', '0').replace('Курит', '1').replace('Бросил(а)', '2'))
    df['Статус Курения'] = df['Статус Курения'].astype(int)
    
    df['Время пробуждения'] = df['Время пробуждения'].apply(lambda x: ''.join(x.split())[:-3])
    df['Время пробуждения'] = df['Время пробуждения'].apply(lambda x: x.replace(':', '.'))
    df['Время пробуждения'] = df['Время пробуждения'].astype(float)
    
    df['Время засыпания'] = df['Время засыпания'].apply(lambda x: ''.join(x.split())[:-3])
    df['Время засыпания'] = df['Время засыпания'].apply(lambda x: x.replace(':', '.'))
    df['Время засыпания'] = df['Время засыпания'].astype(float)
    
    df = pd.merge(df, df_dop1, on='Пол', how='left')
    df = pd.merge(df, df_dop2, on='Семья', how='left')
    df = pd.merge(df, df_dop3, on='Образование', how='left')
    df = pd.merge(df, df_dop4, on='Этнос', how='left')
    df = pd.merge(df, df_dop5, on='Профессия', how='left')
    df = pd.merge(df, df_dop6, on='Национальность', how='left')
    df = pd.merge(df, df_dop7, on='Религия', how='left')
    
    df = df.fillna(-1)

    
    return df

Состояние тренировочного датасета до применения функции

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Data columns (total 39 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     955 non-null    object 
 1   Пол                                    954 non-null    object 
 2   Семья                                  955 non-null    object 
 3   Этнос                                  955 non-null    object 
 4   Национальность                         955 non-null    object 
 5   Религия                                955 non-null    object 
 6   Образование                            955 non-null    object 
 7   Профессия                              955 non-null    object 
 8   Вы работаете?                          955 non-null    int64  
 9   Выход на пенсию                        955 non-null    int64  
 10  Прекращение работы по болезни          955 non-null    int64  
 11  Сахарн

Тренировочный датасет после применения функции.

In [6]:
train = get_clean_df(train, sex, family, education, ethnos, profession, nationality, religion)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 955 entries, 0 to 954
Data columns (total 46 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   ID                                     955 non-null    object 
 1   Пол                                    955 non-null    object 
 2   Семья                                  955 non-null    object 
 3   Этнос                                  955 non-null    object 
 4   Национальность                         955 non-null    object 
 5   Религия                                955 non-null    object 
 6   Образование                            955 non-null    object 
 7   Профессия                              955 non-null    object 
 8   Вы работаете?                          955 non-null    int64  
 9   Выход на пенсию                        955 non-null    int64  
 10  Прекращение работы по болезни          955 non-null    int64  
 11  Сахарн

Найдем статистически значимые колонки при помощи t-критерия Стьюдента в тренировочном датасете для каждого сердечно-сосудистого заболевания, которые позже будут передаваться в машину.

In [7]:
columns_train = [x for x in train.columns if x not in ['Прочие заболевания сердца', 'Сердечная недостаточность', 
                                                       'Стенокардия, ИБС, инфаркт миокарда', 'ОНМК', 'Артериальная гипертензия',
                                                       'ID', 'ID_y', 'Религия', 'Профессия', 'Этнос', 'Семья', 
                                                       'Религия', 'Образование', 'Национальность', 'Пол']]

In [8]:
def ttest_all(column_name, alfa, target, list_name):
    unique = train[column_name].unique()
    combination = list(combinations(unique, 2))

    for i in combination:
        A = train[train[column_name] == i[0]][target]
        B = train[train[column_name] == i[1]][target]

        pvalue = ttest_ind(A, B).pvalue

        if pvalue <= alfa/len(combination): 
            print('Есть статистическая значимость для колонки ', target, ' в колонке ', column_name)
            list_name.append(column_name)
            break

In [9]:
feats_gipertenzia = []

for i in columns_train:
    ttest_all(i, 0.05, 'Артериальная гипертензия', feats_gipertenzia)
    
feats_gipertenzia = feats_gipertenzia[:9]

Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Вы работаете?
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Выход на пенсию
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Сахарный диабет
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Онкология
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Хроническое заболевание легких
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Бронжиальная астма
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Регулярный прим лекарственных средств
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Переломы
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Статус Курения
Есть статистическая значимость для колонки  Артериальная гипертензия  в колонке  Возраст алког
Есть ста

In [10]:
feats_onmk = []

for i in columns_train:
    ttest_all(i, 0.1, 'ОНМК', feats_onmk)
    
feats_onmk = feats_onmk[:5]

Есть статистическая значимость для колонки  ОНМК  в колонке  Вы работаете?
Есть статистическая значимость для колонки  ОНМК  в колонке  Выход на пенсию
Есть статистическая значимость для колонки  ОНМК  в колонке  Прекращение работы по болезни
Есть статистическая значимость для колонки  ОНМК  в колонке  Регулярный прим лекарственных средств
Есть статистическая значимость для колонки  ОНМК  в колонке  Переломы
Есть статистическая значимость для колонки  ОНМК  в колонке  Возраст курения
Есть статистическая значимость для колонки  ОНМК  в колонке  Время пробуждения
Есть статистическая значимость для колонки  ОНМК  в колонке  Сон после обеда
Есть статистическая значимость для колонки  ОНМК  в колонке  пол_цифра
Есть статистическая значимость для колонки  ОНМК  в колонке  образование_цифра


In [11]:
feats_stenokardia = []

for i in columns_train:
    ttest_all(i, 0.05, 'Стенокардия, ИБС, инфаркт миокарда', feats_stenokardia)
    
feats_stenokardia = feats_stenokardia[:7]

Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Вы работаете?
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Выход на пенсию
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Прекращение работы по болезни
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Сахарный диабет
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Туберкулез легких 
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Регулярный прим лекарственных средств
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Переломы
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  Алкоголь
Есть статистическая значимость для колонки  Стенокардия, ИБС, инфаркт миокарда  в колонке  образование_цифра


In [12]:
feats_serd_ned = []

for i in columns_train:
    ttest_all(i, 0.1, 'Сердечная недостаточность', feats_serd_ned)
    
feats_serd_ned = feats_serd_ned[:8]

Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Вы работаете?
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Выход на пенсию
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Прекращение работы по болезни
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Сахарный диабет
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Бронжиальная астма
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Регулярный прим лекарственных средств
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Переломы
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Статус Курения
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Частота пасс кур
Есть статистическая значимость для колонки  Сердечная недостаточность  в колонке  Алкого

In [13]:
feats_prochee = []

for i in columns_train:
    ttest_all(i, 0.01,'Прочие заболевания сердца', feats_prochee)
    
feats_prochee = feats_prochee[:1]

Есть статистическая значимость для колонки  Прочие заболевания сердца  в колонке  Регулярный прим лекарственных средств
Есть статистическая значимость для колонки  Прочие заболевания сердца  в колонке  Возраст алког
Есть статистическая значимость для колонки  Прочие заболевания сердца  в колонке  профессия_цифра


Инициализируем машины для предсказаний каждого из заболеваний.

In [14]:
model_lgbm_gip = lgm.LGBMClassifier(boosting_type = 'gbdt', max_depth=10, n_estimators=200, learning_rate=0.1, 
                                    is_unbalance=True, num_leaves = 50, metric='binary_logloss', objective='binary')

model_lgbm_onmk = lgm.LGBMClassifier(boosting_type = 'gbdt', max_depth=10, n_estimators=200, learning_rate=0.1, 
                                     is_unbalance=True, num_leaves = 50, metric='binary_logloss', objective='binary')

model_lgbm_sten = lgm.LGBMClassifier(boosting_type = 'gbdt', max_depth=10, n_estimators=200, learning_rate=0.1, 
                                     is_unbalance=True, num_leaves = 50, metric='binary_logloss', objective='binary')

model_lgbm_serd_ned = lgm.LGBMClassifier(boosting_type = 'gbdt', max_depth=10, n_estimators=200, learning_rate=0.1, 
                                         is_unbalance=True, num_leaves = 50, metric='binary_logloss', objective='binary')

model_lgbm_prochee = lgm.LGBMClassifier(boosting_type = 'gbdt', max_depth=10, n_estimators=200, learning_rate=0.1, 
                                        is_unbalance=True, num_leaves = 50, metric='binary_logloss', objective='binary')

Разделим обучающий датасет на тренировочный и на валидационный датасеты, чтобы проверить ключевую метрику - recall.

In [15]:
train_gip, val_gip = train_test_split(train, train_size=0.8, random_state=42)

In [16]:
train_gipertenizia = train_gip[feats_gipertenzia]
y_gipertenizia = train_gip['Артериальная гипертензия']

val_gipertenizia = val_gip[feats_gipertenzia]
y_val_gip = val_gip['Артериальная гипертензия']

train_onmk = train_gip[feats_onmk]
y_onmk = train_gip['ОНМК']

val_onmk = val_gip[feats_onmk]
y_val_onmk = val_gip['ОНМК']

train_sten = train_gip[feats_stenokardia]
y_sten = train_gip['Стенокардия, ИБС, инфаркт миокарда']

val_sten = val_gip[feats_stenokardia]
y_val_sten = val_gip['Стенокардия, ИБС, инфаркт миокарда']

train_serd_ned = train_gip[feats_serd_ned]
y_serd_ned = train_gip['Сердечная недостаточность']

val_serd_ned = val_gip[feats_serd_ned]
y_val_serd_ned = val_gip['Сердечная недостаточность']

train_prochee = train_gip[feats_prochee]
y_prochee = train_gip['Прочие заболевания сердца']

val_prochee = val_gip[feats_prochee]
y_val_prochee = val_gip['Прочие заболевания сердца']

In [17]:
for _ in trange(50):
    model_lgbm_gip.fit(train_gipertenizia, y_gipertenizia)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 14.03it/s]


In [18]:
for _ in trange(50):
    gip_pred = model_lgbm_gip.predict(val_gipertenizia)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 399.91it/s]


Recall для артериальной гипертензии.

In [19]:
recall_score(y_val_gip, gip_pred)

0.7191011235955056

In [20]:
for _ in trange(50):
    model_lgbm_onmk.fit(train_onmk, y_onmk)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 22.59it/s]


In [21]:
for _ in trange(50):
    onmk_pred = model_lgbm_onmk.predict(val_onmk)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 458.58it/s]


Recall для ОНМК.

In [22]:
recall_score(y_val_onmk, onmk_pred)

0.625

In [23]:
for _ in trange(50):
    model_lgbm_sten.fit(train_sten, y_sten)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 20.09it/s]


In [24]:
for _ in trange(50):
    sten_pred = model_lgbm_sten.predict(val_sten)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 402.51it/s]


Recall для стенокардии, ИБС и инфаркта миокарда.

In [25]:
recall_score(y_val_sten, sten_pred)

0.64

In [26]:
for _ in trange(50):
    model_lgbm_serd_ned.fit(train_serd_ned, y_serd_ned)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 14.42it/s]


In [27]:
for _ in trange(50):
    serd_ned_pred = model_lgbm_serd_ned.predict(val_serd_ned)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 398.44it/s]


Recall для сердечной недостаточности.

In [28]:
recall_score(y_val_serd_ned, serd_ned_pred)

0.75

In [29]:
for _ in trange(50):
    model_lgbm_prochee.fit(train_prochee, y_prochee)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 46.98it/s]


In [30]:
for _ in trange(50):
    prochee_pred = model_lgbm_prochee.predict(val_prochee)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 480.77it/s]


Recall для прочих заболеваний сердца.

In [31]:
recall_score(y_val_prochee, prochee_pred)

0.8461538461538461

Получим предсказания для тестового датасета.

In [32]:
test = get_clean_df(test, sex, family, education, ethnos, profession, nationality, religion)

In [33]:
test_gipertenizia = test[feats_gipertenzia]

test_onmk = test[feats_onmk]

test_sten = test[feats_stenokardia]

test_serd_ned = test[feats_serd_ned]

test_prochee = test[feats_prochee]

In [34]:
for _ in trange(50):
    test_gip_pred = model_lgbm_gip.predict(test_gipertenizia)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 215.52it/s]


In [35]:
for _ in trange(50):
    test_onmk_pred = model_lgbm_onmk.predict(test_onmk)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 297.62it/s]


In [36]:
for _ in trange(50):
    test_sten_pred = model_lgbm_sten.predict(test_sten)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 250.00it/s]


In [37]:
for _ in trange(50):
    test_serd_ned_pred = model_lgbm_serd_ned.predict(test_serd_ned)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 207.46it/s]


In [38]:
for _ in trange(50):
    test_prochee_pred = model_lgbm_prochee.predict(test_prochee)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 367.65it/s]


In [39]:
shu_pred_gip = pd.DataFrame(data=test_gip_pred, columns=['Артериальная гипертензия'])

shu_pred_onmk = pd.DataFrame(data=test_onmk_pred, columns=['ОНМК'])

shu_pred_sten = pd.DataFrame(data=test_sten_pred, columns=['Стенокардия, ИБС, инфаркт миокарда'])

shu_pred_serd_ned = pd.DataFrame(data=test_serd_ned_pred, columns=['Сердечная недостаточность'])

shu_pred_prochee = pd.DataFrame(data=test_prochee_pred, columns=['Прочие заболевания сердца'])

shu_prediction = test['ID']
shu_prediction = pd.DataFrame(shu_prediction)

shu_prediction = pd.concat([shu_prediction, shu_pred_gip, shu_pred_onmk, shu_pred_sten, shu_pred_serd_ned, shu_pred_prochee], 
                           axis=1)
shu_prediction.head(5)

,ID,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,54-001-019-01,1,1,1,1,1
1,54-002-133-01,1,1,1,1,1
2,54-001-007-01,1,1,1,1,1
3,54-102-116-01,0,0,0,0,0
4,54-502-005-02,1,1,1,1,1
